In [338]:
# ================ SETUP INICIAL ================ #

import sys
import os
import importlib
import pandas as pd
import numpy as np
import re

# Añadimos el path del paquete local 'src/'
sys.path.append(os.path.abspath("../src"))

# Recargamos módulo en caso de cambios recientes
import procesar_zscore
importlib.reload(procesar_zscore)

from procesar_zscore import procesar_archivo, transformar_a_formato_largo

In [340]:
# ================ PARÁMETROS GENERALES ================ #

# Años que forman el panel
years = [2019, 2020, 2021, 2022, 2023]

# Lista vacía para almacenar DataFrames anuales
dfs = []

In [342]:
# ================ PROCESAMIENTO DE ARCHIVOS ================ #

for year in years:
    file_path = f"../data/zscore_sample{year}.csv"
    df_year = procesar_archivo(file_path, year)

    # Verificamos si hay columnas duplicadas en el DataFrame procesado
    duplicated_cols = df_year.columns[df_year.columns.duplicated()].tolist()
    if duplicated_cols:
        print(f" {year} tiene columnas duplicadas: {duplicated_cols}")
    else:
        print(f" {year} columnas únicas")

    dfs.append(df_year)

Columnas disponibles en 2019: ['SP_ENTITY_NAME_2019', 'SP_ENTITY_ID_2019', 'SP_GEOGRAPHY_2019', 'SP_COMPANY_TYPE_2019', 'IQ_INDUSTRY_CLASSIFICATION_2019', 'SP_COMPANY_STATUS_2019', 'IQ_TOTAL_ASSETS_2019', 'IQ_TOTAL_REV_2019', 'IQ_TOTAL_DEBT_2019', 'IQ_INTEREST_EXP_2019', 'IQ_TOTAL_CA_2019', 'IQ_TOTAL_CL_2019', 'IQ_RETAINED_EARNINGS_2019', 'IQ_EBIT_2019', 'IQ_EBITDA_2019', 'SNL_PRETAX_INT_COV_EXCL_AFUDC_2019', 'year']
 2019 columnas únicas
Columnas disponibles en 2020: ['SP_ENTITY_NAME_2020', 'SP_ENTITY_ID_2020', 'SP_GEOGRAPHY_2020', 'SP_COMPANY_TYPE_2020', 'IQ_INDUSTRY_CLASSIFICATION_2020', 'SP_COMPANY_STATUS_2020', 'IQ_TOTAL_ASSETS_2020', 'IQ_TOTAL_REV_2020', 'IQ_TOTAL_DEBT_2020', 'IQ_INTEREST_EXP_2020', 'IQ_TOTAL_CA_2020', 'IQ_TOTAL_CL_2020', 'IQ_RETAINED_EARNINGS_2020', 'IQ_EBIT_2020', 'IQ_EBITDA_2020', 'SNL_PRETAX_INT_COV_EXCL_AFUDC_2020', 'year']
 2020 columnas únicas
Columnas disponibles en 2021: ['SP_ENTITY_NAME_2021', 'SP_ENTITY_ID_2021', 'SP_GEOGRAPHY_2021', 'SP_COMPANY_TYPE_2

In [344]:
# Unimos todos los años en un solo panel longitudinal
df_panel = pd.concat(dfs, axis=0).reset_index(drop=True)

In [345]:
df_panel.replace("", np.nan, inplace=True) # Aseguramos que las celdas vacías sean tratadas como NaN

In [346]:
# Eliminamos filas completamente vacías (excepto 'year')
df_panel = df_panel.dropna(subset=[col for col in df_panel.columns if col != 'year'], how='all').reset_index(drop=True)

In [350]:
[col for col in df_panel.columns if "dup" in col]

[]

In [352]:
# ================ LIMPIEZA DE VARIABLES NUMÉRICAS (IQ_) ================ #

# Creamos función para limpieza numérica

def limpiar_columna_numerica(col):
    """
    Transforma columnas con números en formato texto (contable) a float:
    - Convierte (1,234) en -1234
    - Elimina comas, espacios
    - Reemplaza 'NM', '--', '', etc. por np.nan
    """
    col_limpia = (
        col.astype(str) # Primero convertimos todo a texto para hacer modificaciones
           .str.replace(",", "", regex=False)
           .str.replace("(", "-", regex=False)
           .str.replace(")", "", regex=False)
           .str.replace(r"\b(NM|NA|--|n/a|N/A)\b", "", regex=True) # Aplicamos raw string para valores no numéricos comunes en bases de datos
           .str.strip()
    )
    return pd.to_numeric(col_limpia.replace("", np.nan), errors="coerce") # Reemplazamos valores vacíos y lo devolvemos a valor numérico y, si hay error, lo hacemos NaN

In [354]:
# Filtramos columnas IQ_ que contienen al menos un valor convertible a número
cols_numericas = [
    col for col in df_panel.columns
    if col.startswith("IQ_") and df_panel[col].str.replace(",", "").str.replace("(", "").str.replace(")", "").str.strip().str.match(r"^-?\d+(\.\d+)?$").sum() > 0
]

In [356]:
# Aplicamos limpieza a todas las columnas IQ_
for col in cols_numericas:
    df_panel[col] = limpiar_columna_numerica(df_panel[col])

In [357]:
# ========== CONVERSIÓN A FORMATO LARGO ========== #

df_panel_largo = transformar_a_formato_largo(df_panel)

In [359]:
# ================ CÁLCULO DE RATIOS PARA Z-SCORE ================ #

# Creamos Working Capital primero
df_panel_largo["IQ_WORKING_CAPITAL"] = df_panel_largo["IQ_TOTAL_CA"] - df_panel_largo["IQ_TOTAL_CL"]

# X1: Working Capital / Total Assets
df_panel_largo["Z_X1_WORKING_CAPITAL_RATIO"] = df_panel_largo["IQ_WORKING_CAPITAL"] / df_panel_largo["IQ_TOTAL_ASSETS"]

# X2: Retained Earnings / Total Assets
df_panel_largo["Z_X2_RE_RATIO"] = df_panel_largo["IQ_RETAINED_EARNINGS"] / df_panel_largo["IQ_TOTAL_ASSETS"]

# X3: EBIT / Total Assets
df_panel_largo["Z_X3_EBIT_RATIO"] = df_panel_largo["IQ_EBIT"] / df_panel_largo["IQ_TOTAL_ASSETS"]

# X5: Sales / Total Assets
df_panel_largo["Z_X5_SALES_RATIO"] = df_panel_largo["IQ_TOTAL_REV"] / df_panel_largo["IQ_TOTAL_ASSETS"]

# Confirmamos que las nuevas columnas se hayan creado
print("✅ Ratios Z-Score generados:")
print(df_panel_largo.filter(regex="^Z_").columns.tolist())

✅ Ratios Z-Score generados:
['Z_X1_WORKING_CAPITAL_RATIO', 'Z_X2_RE_RATIO', 'Z_X3_EBIT_RATIO', 'Z_X5_SALES_RATIO']


In [362]:
# ====================== FILTROS FINALES ====================== #

# Indicador si la empresa tiene los 4 ratios completos
df_panel_largo["datos_completos_zscore"] = df_panel_largo.filter(regex="^Z_").notna().all(axis=1)

# Agregamos un flag para activos "confiables" (usamos >= 1,000 como umbral mínimo para evitar ratios explosivos)
df_panel_largo["activo_valido"] = df_panel_largo["IQ_TOTAL_ASSETS"] >= 1000

# Creamos una columna que indique si es un registro elegible para análisis de Z-Score
df_panel_largo["registro_valido_modelo"] = (
    df_panel_largo["datos_completos_zscore"] & 
    df_panel_largo["activo_valido"]
)

# Reportamos cuántos registros son válidos
total_validos = df_panel_largo["registro_valido_modelo"].sum()
print(f"✅ Registros válidos para modelar Z-Score: {total_validos:,} de {len(df_panel_largo):,} ({total_validos/len(df_panel_largo):.2%})")

✅ Registros válidos para modelar Z-Score: 64,722 de 198,360 (32.63%)


In [364]:
# Sustituimos valores infinitos por NaN
df_panel_largo.replace([np.inf, -np.inf], np.nan, inplace=True)

In [366]:
# Indicador de cuántos ratios Z fueron posibles de calcular por fila
df_panel_largo["zscore_valid_ratios"] = df_panel_largo.filter(regex="^Z_").notna().sum(axis=1)

In [368]:
# Filtro de outliers extremos en EBIT / Assets
df_panel_largo = df_panel_largo[
    (df_panel_largo["Z_X3_EBIT_RATIO"].isna()) |
    ((df_panel_largo["Z_X3_EBIT_RATIO"] < 10) & (df_panel_largo["Z_X3_EBIT_RATIO"] > -5))
]

In [370]:
# ========== EXPORTACIÓN ========== #

df_panel_largo.to_csv("../outputs/panel_zscore_largo.csv", index=False)
print("✅ Archivo exportado como: panel_zscore_largo.csv")

✅ Archivo exportado como: panel_zscore_largo.csv


In [200]:
# ========== VALIDACIÓN POSTERIOR ========== #

# Verificamos estructura
print(df_panel_largo.shape)
print(df_panel_largo.dtypes)

# Ejemplo de empresas con más valores nulos
nulls = df_panel_largo.isna().mean().sort_values(ascending=False)
print("\nColumnas con más nulos:\n", nulls.head(10))

# Vista previa
df_panel_largo.head()

(991800, 17)
year                               int64
SP_ENTITY_NAME                    object
SP_ENTITY_ID                      object
SP_GEOGRAPHY                      object
SP_COMPANY_TYPE                   object
IQ_INDUSTRY_CLASSIFICATION        object
SP_COMPANY_STATUS                 object
IQ_TOTAL_ASSETS                  float64
IQ_TOTAL_REV                     float64
IQ_TOTAL_DEBT                    float64
IQ_INTEREST_EXP                  float64
IQ_TOTAL_CA                      float64
IQ_TOTAL_CL                      float64
IQ_RETAINED_EARNINGS             float64
IQ_EBIT                          float64
IQ_EBITDA                        float64
SNL_PRETAX_INT_COV_EXCL_AFUDC     object
dtype: object

Columnas con más nulos:
 SNL_PRETAX_INT_COV_EXCL_AFUDC    0.998591
IQ_INTEREST_EXP                  0.937284
IQ_EBITDA                        0.935529
IQ_EBIT                          0.930044
IQ_RETAINED_EARNINGS             0.911114
IQ_TOTAL_REV                     0.90839

,year,SP_ENTITY_NAME,SP_ENTITY_ID,SP_GEOGRAPHY,SP_COMPANY_TYPE,IQ_INDUSTRY_CLASSIFICATION,SP_COMPANY_STATUS,IQ_TOTAL_ASSETS,IQ_TOTAL_REV,IQ_TOTAL_DEBT,IQ_INTEREST_EXP,IQ_TOTAL_CA,IQ_TOTAL_CL,IQ_RETAINED_EARNINGS,IQ_EBIT,IQ_EBITDA,SNL_PRETAX_INT_COV_EXCL_AFUDC
0,2019,"""AGBank"" OJSC",4538223,Europe,Private Company,Financials,Liquidating,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,"""AMIO BANK"" CJSC",4559124,Europe,Private Company,Financials,Operating Subsidiary,1538862.0,31092.0,118122.0,NaN,310153.0,1358825.0,6467.0,NaN,NaN,NaN
2,2019,"""Bank Dabrabyt"" Joint-stock Company",4265923,Europe,Private Company,Financials,Operating,594611.0,38160.0,41650.0,NaN,181152.0,524820.0,34058.0,NaN,NaN,NaN
3,2019,"""DBK-Leasing"" JSC",4414560,Europe,Private Company,Financials,Operating Subsidiary,1185982.0,32798.0,661782.0,NaN,788350.0,7370.0,2303.0,NaN,NaN,NaN
4,2019,"""Muganbank"" Open Joint Stock Company",4552311,Europe,Private Company,Financials,Operating,355389.0,-295.0,115385.0,NaN,114803.0,218986.0,-39841.0,NaN,NaN,NaN


In [318]:
print(df_panel_largo.columns.tolist())

['year', 'SP_ENTITY_NAME', 'SP_ENTITY_ID', 'SP_GEOGRAPHY', 'SP_COMPANY_TYPE', 'IQ_INDUSTRY_CLASSIFICATION', 'SP_COMPANY_STATUS', 'IQ_TOTAL_ASSETS', 'IQ_TOTAL_REV', 'IQ_TOTAL_DEBT', 'IQ_INTEREST_EXP', 'IQ_TOTAL_CA', 'IQ_TOTAL_CL', 'IQ_RETAINED_EARNINGS', 'IQ_EBIT', 'IQ_EBITDA', 'SNL_PRETAX_INT_COV_EXCL_AFUDC', 'IQ_WORKING_CAPITAL', 'Z_X1_WORKING_CAPITAL_RATIO', 'Z_X2_RE_RATIO', 'Z_X3_EBIT_RATIO', 'Z_X5_SALES_RATIO']
